In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=66924a5aa411ee82ed7c5fffb8f54d3d8b58f8fd5e49247c276569e9a0783862
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, min

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder.appName("California Housing Analysis").getOrCreate()

In [ ]:
# (Assuming that the dataset is available locally in the sample_data directory)
df = spark.read.csv("/content/housing.csv", header=True, inferSchema=True)



In [ ]:
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [ ]:
# Register DataFrame as Temp Table
df.createOrReplaceTempView("california_housing")

In [ ]:
# Example Analysis 1: Calculate average house value by median income bracket
result1_shikha = spark.sql("""
    SELECT median_income, AVG(median_house_value) as avg_house_value
    FROM california_housing
    GROUP BY median_income
    ORDER BY median_income""")

In [ ]:
result1_shikha.show()

+-------------+---------------+
|median_income|avg_house_value|
+-------------+---------------+
|       0.4999|       163608.5|
|        0.536|       166999.9|
|       0.5495|        91700.0|
|       0.6433|       111300.0|
|       0.6775|       350000.0|
|       0.6825|       187500.0|
|       0.6831|        87500.0|
|        0.696|        42500.0|
|       0.6991|        89500.0|
|       0.7007|       134400.0|
|       0.7025|       500001.0|
|       0.7054|       137500.0|
|       0.7068|       200000.0|
|       0.7069|        70300.0|
|       0.7075|        78800.0|
|        0.716|       104200.0|
|       0.7235|        71300.0|
|       0.7286|        95200.0|
|       0.7403|        68600.0|
|       0.7445|       112500.0|
+-------------+---------------+
only showing top 20 rows



In [ ]:
# Example Analysis 2: Find the maximum and minimum house values in each housing block
result2_shikha = spark.sql("""
    SELECT longitude, latitude, MAX(median_house_value) as max_house_value,
           MIN(median_house_value) as min_house_value
    FROM california_housing
    GROUP BY longitude, latitude
    ORDER BY max_house_value DESC
    LIMIT 10""")

In [ ]:
result2_shikha.show()

+---------+--------+---------------+---------------+
|longitude|latitude|max_house_value|min_house_value|
+---------+--------+---------------+---------------+
|  -118.43|   34.02|       500001.0|       500001.0|
|  -118.57|   34.27|       500001.0|       500001.0|
|  -118.12|   33.76|       500001.0|       461500.0|
|  -121.74|   37.35|       500001.0|       500001.0|
|  -122.06|    37.3|       500001.0|       500001.0|
|  -118.07|   33.72|       500001.0|       485000.0|
|  -122.03|   37.83|       500001.0|       500001.0|
|  -121.91|   37.81|       500001.0|       500001.0|
|  -119.23|   34.19|       500001.0|       500001.0|
|  -118.35|   34.15|       500001.0|       500001.0|
+---------+--------+---------------+---------------+



In [ ]:
# Example Analysis 3: Determine the average house value for houses older than 50 years
result3_shikha = spark.sql("""
    SELECT AVG(median_house_value) as avg_old_house_value
    FROM california_housing
    WHERE housing_median_age > 50""")

In [ ]:
result3_shikha.show()

+-------------------+
|avg_old_house_value|
+-------------------+
|  273687.5647236942|
+-------------------+



In [ ]:
spark.stop()

In [ ]:
spark = SparkSession.builder.appName("California Housing Analysis Advance").getOrCreate()

In [ ]:
df = spark.read.csv("/content/housing.csv", header=True, inferSchema=True)



In [ ]:
df.createOrReplaceTempView("california_housing")

In [ ]:
# 1. Average House Value by Proximity to Ocean (Based on Longitude)
result1_shikha = spark.sql("""
    SELECT CASE
               WHEN longitude < -122 THEN 'Near Ocean'
               ELSE 'Far from Ocean'
           END as proximity_to_ocean,
           AVG(median_house_value) as avg_house_value
    FROM california_housing
    GROUP BY proximity_to_ocean""")

In [ ]:
result1_shikha.show()

+------------------+------------------+
|proximity_to_ocean|   avg_house_value|
+------------------+------------------+
|        Near Ocean| 244230.8817746408|
|    Far from Ocean|197963.18317039526|
+------------------+------------------+



In [ ]:
# 2. Average Number of Rooms by Age of Housing
result2_shikha = spark.sql("""
    SELECT housing_median_age,
           AVG(total_rooms) as avg_rooms
    FROM california_housing
    GROUP BY housing_median_age
    ORDER BY housing_median_age""")

In [ ]:
result2_shikha.show()

NameError: name 'result2_shikha' is not defined

In [ ]:
# 3. Top 10 Locations with the Highest Median Income
result3_shikha = spark.sql("""
    SELECT longitude, latitude, median_income
    FROM california_housing
    ORDER BY median_income DESC
    LIMIT 10""")

In [ ]:
result3_shikha.show()

+---------+--------+-------------+
|longitude|latitude|median_income|
+---------+--------+-------------+
|  -118.32|   34.06|      15.0001|
|  -118.34|   34.08|      15.0001|
|  -118.49|   34.06|      15.0001|
|   -118.4|   34.11|      15.0001|
|  -118.33|   34.07|      15.0001|
|  -118.44|   34.09|      15.0001|
|  -118.33|   34.06|      15.0001|
|  -118.43|   34.08|      15.0001|
|  -121.96|   37.74|      15.0001|
|  -118.33|   34.07|      15.0001|
+---------+--------+-------------+



In [ ]:
# 4. Relationship Between Population and House Value (Correlation Analysis)
result4_shikha = spark.sql("""
    SELECT POPULATION, median_house_value
    FROM california_housing""")
result4_shikha.corr("population", "median_house_value")

-0.024649678888894865

In [ ]:
# 5. Average House Value by Income Level (Low, Medium, High)
result5_shikha = spark.sql("""
    SELECT CASE
               WHEN median_income < 2.5 THEN 'Low Income'
               WHEN median_income BETWEEN 2.5 AND 4.5 THEN 'Medium Income'
               ELSE 'High Income'
           END as income_level,
           AVG(median_house_value) as avg_house_value
    FROM california_housing
    GROUP BY income_level""")

In [ ]:
result5_shikha.show()

+-------------+------------------+
| income_level|   avg_house_value|
+-------------+------------------+
|  High Income| 305672.1538076987|
|   Low Income|121350.76826460121|
|Medium Income|188143.66720746298|
+-------------+------------------+



In [ ]:
# 6. Number of Houses in Different Age Groups
result6_shikha = spark.sql("""
    SELECT CASE
               WHEN housing_median_age < 20 THEN 'New'
               WHEN housing_median_age BETWEEN 20 AND 40 THEN 'Middle-aged'
               ELSE 'Old'
           END as age_group,
           COUNT(*) as house_count
    FROM california_housing
    GROUP BY age_group
    ORDER BY house_count DESC""")

In [ ]:
result6_shikha.show()

+-----------+-----------+
|  age_group|house_count|
+-----------+-----------+
|Middle-aged|      10934|
|        New|       5828|
|        Old|       3878|
+-----------+-----------+



In [ ]:
# 7. Average Number of Bedrooms Per House
result7_shikha = spark.sql("""
    SELECT AVG(total_bedrooms / households) as avg_bedrooms_per_house
    FROM california_housing""")

In [ ]:
result7_shikha.show()

+----------------------+
|avg_bedrooms_per_house|
+----------------------+
|    1.0970623858069932|
+----------------------+



In [ ]:
# 8. Median House Value by Latitude and Longitude
result8_shikha = spark.sql("""
    SELECT latitude, longitude,
           PERCENTILE_APPROX(median_house_value, 0.5) as median_value
    FROM california_housing
    GROUP BY latitude, longitude
    ORDER BY median_value DESC
    LIMIT 10""")

In [ ]:
result8_shikha.show()

+--------+---------+------------+
|latitude|longitude|median_value|
+--------+---------+------------+
|   32.83|  -117.31|    500001.0|
|   32.71|  -117.24|    500001.0|
|   32.86|  -117.25|    500001.0|
|   32.83|  -117.26|    500001.0|
|   32.85|  -117.26|    500001.0|
|   32.83|  -117.25|    500001.0|
|   32.81|  -117.29|    500001.0|
|   32.84|  -117.25|    500001.0|
|   32.82|  -117.26|    500001.0|
|   32.85|   -117.3|    500001.0|
+--------+---------+------------+



In [ ]:
# 9. Households with More Than 4 Persons Per Household
result9_shikha = spark.sql("""
    SELECT COUNT(*) as large_households
    FROM california_housing
    WHERE population / households > 4""")

In [ ]:
result9_shikha.show()


+----------------+
|large_households|
+----------------+
|            1741|
+----------------+



In [ ]:
# 10. Areas with the Highest House Density (Rooms per Area)
result10_shikha = spark.sql("""
    SELECT latitude, longitude, (total_rooms / housing_median_age) as rooms_density
    FROM california_housing
    ORDER BY rooms_density DESC
    LIMIT 10""")

In [ ]:
result10_shikha.show()

+--------+---------+------------------+
|latitude|longitude|     rooms_density|
+--------+---------+------------------+
|   33.58|   -117.2|           15225.0|
|   38.43|  -121.44|13106.666666666666|
|   38.72|  -121.35|           10948.5|
|   33.57|  -117.16|           10195.5|
|   33.89|  -117.74|           9484.25|
|   33.89|  -117.52|            8989.0|
|   33.87|  -117.64|            8735.0|
|   33.52|  -117.12|           7600.25|
|    33.9|  -117.19|            7020.0|
|   38.77|  -121.33|            6738.0|
+--------+---------+------------------+



In [ ]:
spark.stop()